## Recomendador básico

Instalar el Frameword Surprise

In [25]:
!pip install numpy
!pip install scikit-surprise

Importamos las bibliotecas necesarias para nuestro recomendador

In [26]:
# Cargamos las librerías necesarias de Pandas
import pandas as pd
import numpy as np

In [27]:
# Cargamos las librerías necesarias de Surprise
from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import accuracy

Cargamos el dataset en pandas para luego pasarlo a surprise

In [28]:
# Cargamos el CSV con el dataframe de Pandas
df = pd.read_csv('songs_dataset.csv')

reader = Reader(rating_scale=(1, 30))

# Vamos a transformar los distintos géneros a números para que el recomendador funcione por recomendación por género
# Con esto evitamos los repetidos
distinct_genres = df['top genre'].unique()

# Asignamos números a los géneros
value_to_number = {genre: i+1 for i, genre in enumerate(distinct_genres)}

# Creamos una nueva columna que es la misma que géneros pero en números
df['genre number'] = df['top genre'].map(value_to_number)

# Load the Pandas DataFrame into a Surprise Dataset
data = Dataset.load_from_df(df[['artist', 'title', 'genre number']], reader)

In [107]:
df

,id,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,genre number
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83,1
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82,2
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80,3
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79,3
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,599,Find U Again (feat. Camila Cabello),Mark Ronson,dance pop,2019,104,66,61,-7,20,16,176,1,3,75,3
599,600,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,pop,2019,95,79,75,-6,7,61,206,21,12,75,4
600,601,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,dance pop,2019,136,76,53,-5,9,65,260,7,34,70,3
601,602,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,dance pop,2019,114,79,60,-6,42,24,217,1,7,69,3


Entrenar los datos

In [108]:
# Crear un modelo modelo k-NN
sim_options = {
    'name': 'cosine',  
    'user_based': False  # Recomendador basado en contenidos
}
clf = KNNBasic(sim_options=sim_options, k=50, verbose=True)

In [ ]:
#Measures probar con RMSE o MAE, cuál sea el mejor
cv = cross_validate(clf, data, measures=['RMSE', 'MAE'], cv=7, verbose=True)

Función para determinar canciones similares

In [ ]:
#Para hacer la búsqueda si la canción está en los datos entrenados
items_ids = [clf.trainset.to_raw_iid(iid) for iid in clf.trainset.all_items()]

In [ ]:
def get_similar_songs(song_title, k=10):
    try: 
        # Comprueba si la canción está en los datos entrenados
        if song_title not in items_ids:
            return []  # Devuelve una lista vacía si no está entre los datos entrenados

        # Obtiene el id de la canción para luego pasarlo al algoritmo
        song_id = clf.trainset.to_inner_iid(song_title)

        # Usa el modelo k-NN para encontrar canciones similares
        similar_items = clf.get_neighbors(song_id, k)

        # Una vez tiene los ids de las canciones similares se pasa a los nombres reales de las canciones
        similar_songs = [clf.trainset.to_raw_iid(item_id) for item_id in similar_items]

        return similar_songs

    except IndexError as e:
        return []

Generar una "playlist" random con 10 canciones

In [ ]:
import random

random_songs = df.sample(n=10)

In [ ]:
random_songs

Conseguir las recomendaciones 

In [ ]:
similar_songs = []

for song_title in random_songs['title']:
    sim_songs = get_similar_songs(song_title, k=10)    
    
    #Para evitar canciones ya recomendadas antes
    for s in sim_songs:
        if s not in similar_songs:
            similar_songs.append(s)

In [ ]:
similar_songs

Vamos a probar recomendación con una canción

In [ ]:
similar_songs = get_similar_songs('Find U Again (feat. Camila Cabello)', k=10)
similar_songs

#### Recomendador sin surprise
Dado una canción o el nombre de un artista, se recomienda más canciones del artista (en el caso de por canción no se recomienda esa canción)

In [29]:
df = df.sort_values(by='pop') # ordena por popularidad
df2_grouped = df.groupby('artist') # pone las canciones del mismo artista juntos
df3_grouped = df.groupby('top genre') # por géneros porque en el caso de que no haya suficientes del mismo artista será por las canciones más populares del género

df2 = pd.DataFrame()
for key, group in df2_grouped:
    df2 = pd.concat([df2, group], axis=0)

df3 = pd.DataFrame()
for key, group in df3_grouped:
    df3 = pd.concat([df3, group], axis=0)

In [30]:
df2.loc[df2['top genre'] == "dance pop"]

,id,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,genre number
31,32,My First Kiss - feat. Ke$ha,3OH!3,dance pop,2010,138,89,68,-4,36,83,192,1,8,62,3
546,547,Supernova,Ansel Elgort,dance pop,2018,140,62,77,-5,21,68,191,6,10,72,3
323,324,Focus,Ariana Grande,dance pop,2015,100,88,67,-6,44,79,211,27,24,66,3
171,172,The Way,Ariana Grande,dance pop,2013,82,88,65,-3,8,86,227,29,11,68,3
291,292,Break Free,Ariana Grande,dance pop,2015,130,70,69,-5,20,28,215,1,5,75,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,546,Let Me,ZAYN,dance pop,2018,168,57,46,-6,10,19,185,24,5,72,3
457,458,I Don’t Wanna Live Forever (Fifty Shades Darker),ZAYN,dance pop,2017,118,45,74,-8,33,9,245,6,6,78,3
517,518,Dusk Till Dawn - Radio Edit,ZAYN,dance pop,2018,180,44,26,-7,11,10,239,10,4,83,3
286,287,Never Forget You,Zara Larsson,dance pop,2015,146,73,58,-6,27,28,213,0,5,77,3


In [31]:
def recommender_by_artist(artist, song_excl):
    songs = df2.loc[df2['artist'] == artist]
    if song_excl != None: 
        songs_ret = songs.loc[songs['title'] != song_excl, ['id', 'title', 'artist', 'top genre']]
        return songs_ret
    else: 
        return songs[['id', 'title', 'artist', 'top genre']]

In [119]:
def recommender_by_genre(genre, song_excl):
    songs = df3.loc[df3['top genre'] == genre]
    if song_excl != None: 
        songs_ret = songs.loc[songs['title'] != song_excl, ['id', 'title', 'artist', 'top genre']]
        return songs_ret
    else: 
        return songs[['id', 'title', 'artist', 'top genre']]

In [32]:
def get_id(songs):
    return songs['id']  

In [38]:
def recommender_manual(song_artist):
    if song_artist in df['title'].values: #lo que se ha pasado por parámetro es el título de la canción
        fil = df.loc[df['title'] == song_artist]
        artist = fil['artist'].values[0]
        
        songs = recommender_by_artist(artist, song_artist)

        if len(songs) < 10:
            genre = fil['top genre'].values[0] 
            songs_genre = recommender_by_genre(genre, song_artist)
            
            songs_ret = pd.concat([songs, songs_genre])[:10]

            return get_id(songs_ret)
        
        elif len(songs) > 10:
            return get_id(songs[:10])
        
        else:
            return get_id(songs)

    elif song_artist in df['artist'].values: #se ha pasado por parámetro el nombre del artista
        songs = recommender_by_artist(song_artist, None)

        return get_id(songs[:10]) #no hay genero asociado por artista, lo que muestre si hay 15 o menos del artista
    
    else: #no exista tal artista o canción
        print("Error: no existe tal artista o canción en la base de datos\n")
        return []

In [34]:
def recommender_no_surprise(songs_arists):
    songs = []
    for song_artist in songs_arists:
        songs_aux = recommender_manual(song_artist)
        songs.extend(songs_aux)
    
    return songs

In [39]:
recommender_manual("#thatPOWER")

138    139
362    363
267    268
103    104
361    362
102    103
101    102
100    101
266    267
99     100
Name: id, dtype: int64

In [40]:
recommender_manual('Rihanna')

205    206
92      93
42      43
135    136
91      92
133    134
332    333
85      86
126    127
72      73
Name: id, dtype: int64

In [41]:
recommender_no_surprise(["#thatPOWER", "Rihanna"])

[139,
 363,
 268,
 104,
 362,
 103,
 102,
 101,
 267,
 100,
 206,
 93,
 43,
 136,
 92,
 134,
 333,
 86,
 127,
 73]